In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from scipy import signal
#import xesmf as xe
from scipy.ndimage import gaussian_filter
from scipy.stats.stats import pearsonr


In [ ]:
#define function to get all the data at once, use same years for climatology for all data
def get_data():
    
    #climatology years
#    cyr1,cyr2='1993-01-01','2019-12-31'
    cyr1,cyr2='2010-01-01','2019-12-31'
    
    # CCMP test
    dir_pattern_zarr = 'F:/data/sat_data/ccmp/zarr/'
    ds= xr.open_zarr(dir_pattern_zarr)
    ds = ds.rename({'latitude':'lat','longitude':'lon'})
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds = ds.sortby(ds.lon)
    ds = ds.sel(time=slice(cyr1,cyr2))
    ds_ccmp = ds.drop('nobs')
    # MAKE DAY AVERAGE
    ds_ccmp = ds_ccmp.resample(time='D').mean()
    for var in ds_ccmp:
        tem = ds_ccmp[var].attrs
        tem['var_name']='ccmp_'+str(var)
        ds_ccmp[var].attrs=tem
    ds_ccmp_clim = ds_ccmp.sel(time=slice(cyr1,cyr2))
    ds_ccmp_clim = ds_ccmp_clim.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
    
    #sst
    dir_pattern_zarr = 'F:/data/sst/cmc/zarr/'
    ds_sst= xr.open_zarr(dir_pattern_zarr)
    ds_sst = ds_sst.drop({'analysis_error','mask','sea_ice_fraction'})
    ds_sst = ds_sst.sel(time=slice(cyr1,cyr2))
    tem = ds_sst.analysed_sst.attrs
    tem['var_name']='cmc_sst'
    ds_sst.analysed_sst.attrs=tem
    ds_sst_clim = ds_sst.sel(time=slice(cyr1,cyr2))
    ds_sst_clim = ds_sst_clim.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
    
    #put data into a dictionary
    data_dict={'wnd':ds_ccmp,
               'sst':ds_sst}
    clim_dict={'wnd_clim':ds_ccmp_clim,
               'sst_clim':ds_sst_clim}
  
    return data_dict,clim_dict


def multi_apply_along_axis(func1d, axis, arrs, *args, **kwargs):
    """
    function from: https://climate-cms.org/2019/07/29/multi-apply-along-axis.html
    Given a function `func1d(A, B, C, ..., *args, **kwargs)`  that acts on 
    multiple one dimensional arrays, apply that function to the N-dimensional
    arrays listed by `arrs` along axis `axis`
    
    If `arrs` are one dimensional this is equivalent to::
    
        func1d(*arrs, *args, **kwargs)
    
    If there is only one array in `arrs` this is equivalent to::
    
        numpy.apply_along_axis(func1d, axis, arrs[0], *args, **kwargs)
        
    All arrays in `arrs` must have compatible dimensions to be able to run
    `numpy.concatenate(arrs, axis)`
    
    Arguments:
        func1d:   Function that operates on `len(arrs)` 1 dimensional arrays,
                  with signature `f(*arrs, *args, **kwargs)`
        axis:     Axis of all `arrs` to apply the function along
        arrs:     Iterable of numpy arrays
        *args:    Passed to func1d after array arguments
        **kwargs: Passed to func1d as keyword arguments
    """
    import numpy
    # Concatenate the input arrays along the calculation axis to make one big
    # array that can be passed in to `apply_along_axis`
    carrs = numpy.concatenate(arrs, axis)
    
    # We'll need to split the concatenated arrays up before we apply `func1d`,
    # here's the offsets to split them back into the originals
    offsets=[]
    start=0
    for i in range(len(arrs)-1):
        start += arrs[i].shape[axis]
        offsets.append(start)
            
    # The helper closure splits up the concatenated array back into the components of `arrs`
    # and then runs `func1d` on them
    def helperfunc(a, *args, **kwargs):
        arrs = numpy.split(a, offsets)
        return func1d(*[*arrs, *args], **kwargs)
    
    # Run `apply_along_axis` along the concatenated array
    return numpy.apply_along_axis(helperfunc, axis, carrs, *args, **kwargs)


In [ ]:
data,clim=get_data()

In [ ]:
data['sst']

In [ ]:
data['wnd']

In [ ]:
ds_sst, ds_wnd = data['sst'],data['wnd']
ds_wnd['wspd']=(ds_wnd.uwnd**2+ds_wnd.vwnd**2)**.5
#interp doesn't work on chunked dims so rechunk
ds_sst = ds_sst.chunk({'time':1,'lat':901,'lon':1800})
ds_new = ds_sst.interp(lat = ds_wnd.lat,lon=ds_wnd.lon)

In [ ]:
#SST
#remove mean, seasonal cycle, trend before analysis
clim = ds_new.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
ds_new_tem = ds_new.groupby('time.dayofyear')-clim
#detrending doesn't work with nan so fill with nan but will have to mask at end and pixels with nan in timeseries
tem = ds_new_tem.fillna(0)
ds_detrended_sst = signal.detrend(tem.analysed_sst,axis=0)

In [ ]:
ds_new_tem.analysed_sst.isel(lon=720,lat=310).plot()
plt.plot(ds_new_tem.time,ds_detrended_sst[:,310,720])

In [ ]:
#WSPD
#remove mean, seasonal cycle, trend before analysis
clim = ds_wnd.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
ds_new_tem = ds_wnd.groupby('time.dayofyear')-clim
#detrending doesn't work with nan so fill with nan but will have to mask at end and pixels with nan in timeseries
tem = ds_new_tem.fillna(0)
ds_detrended_wnd = signal.detrend(tem.wspd,axis=0)
ds_new_tem.wspd.isel(lon=720,lat=310).plot()
plt.plot(ds_new_tem.time,ds_detrended_wnd[:,310,720])

In [ ]:
corr = multi_apply_along_axis(pearsonr, 0, [ds_detrended_sst,ds_detrended_wnd])
print(corr.shape)
print(type(corr))
#not sure why by 1440 is blank
acorr = corr[0,:,:]
acorr2 = np.concatenate([acorr[:,720:1439],acorr[:,:1439],acorr[:,:720]],axis=1)
#plt.imshow(acorr[:,:1439],vmin=-.1,vmax=.1,cmap='RdBu')
#plt.imshow(acorr2,vmin=-.1,vmax=.1,cmap='viridis')
data = acorr2
data = np.nan_to_num(data,0)
lowpass = ndimage.gaussian_filter(data, 40)
gauss_highpass = data - lowpass




In [ ]:
plt.imshow(lowpass,vmin=-.1,vmax=.1,cmap='RdBu')
plt.imshow(gauss_highpass,vmin=-.1,vmax=.1,cmap='RdBu')